# Глубинное обучение для текстовых данных, ФКН ВШЭ
## Домашнее задание 4: Direct Preference Optimization 

__Мягкий дедлайн 16.11.25 23:59__ \
__Жесткий дедлайн 19.11.25 23:59__

### О задании

В этом задании вам предстоит обучить большую LLM для ответов на вопросы с помощью DPO, а также реализовать LoRA для эффективного обучения. 

### Оценивание и штрафы

Максимально допустимая оценка за работу — __11 баллов__.

Оценка за это домашнее задание будет формироваться из оценки за __задания__ и за __отчет__, в котором от вас требуется написать о проделанной работе. За отчет можно получить до 2-х баллов, однако в случае отсутствия отчета баллы за соответствующие задания не будут ставиться. Мы настаиваем на том, чтобы вы оформили весь код в виде полноценного проекта. Этот ноутбук нужно рассматривать скорее как файл с условием, чем как место для написания массивного кода. За сдачу больших ноутбуков с кодом оценка будет снижена. Ответы на все вопросы в заданиях можно (нужно) писать в отчете.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

### План решения

<img src="https://miro.medium.com/v2/resize:fit:1400/1*lK6iJMz5CGh2fo7TsDn15A.png" alt="drawing" width="700"/>

Обучение следованию инструкциям с помощью DPO разбивается на два этапа:    
1. __Supervised Fine-tuning (SFT)__ – обучение базовой модели ответам на запросы в нужном формате.
2. __Direct Preference Optimization (DPO)__ – обучение SFT модели приоритизации "хороших" ответов.

Мы не хотим обучать модели целиком по двум причинам: 1) используемые модели очень большие; 2) нам требуется лишь выравнить модель с нашими предпочтениями, не внося в нее новых знаний, что не требует серьезного обучения. Поэтому мы будем использовать PEFT, а именно LoRA для обучения.

Таким образом, вам надо будет:
1. Реализовать и протестировать LoRA
2. Разобраться с данными и привести их к нужному формату
3. Обучить SFT модель
4. Обучить DPO модель
5. Порадоваться, что вы молодцы и со всем справились
6. (Опционально) сделать веб-интерфейс для вашей модели, переиспользуя код из первой домашки (мы можем выдать бонусы, если получится классно).

### О датасете

Мы будем работать с датасетом [Anthropic Helpful-Harmless](https://huggingface.co/datasets/Anthropic/hh-rlhf) для RLHF. В нем содержится 160к примеров ответов на вопросы с историей.

### Low-Rank Adaptation (LoRA)

<img src="https://heidloff.net/assets/img/2023/08/lora.png" alt="drawing" width="600"/>

__Задание 1 (3 балла).__ Реализуйте самостоятельно модуль LoRA для эффективного обучения LLM по схеме, описанной в [статье](https://arxiv.org/pdf/2106.09685). Встройте его в свою любимую LLM и убедитесь, что ошибка убывает при обучении параметров LoRA на безусловную генерацию. Для этого возьмите любые данные на свой выбор. Замерьте насколько уменьшилось число обучаемых параметров, как изменилась скорость во время forward и backward процессов и как изменились затраты по памяти. Сделайте выводы и напишите о них в отчете.

In [ ]:
from utils import get_tokenizer, get_dataloaders, count_trinable_params
from lora import get_base_model, get_lora_model, train_model_exp, eval_model_exp
from sft import get_sft_dataloaders, train_model_sft, eval_model_sft

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import torch
import torch.nn as nn
from transformers import GPT2Config, GPT2Model, GPT2Tokenizer, RobertaPreTrainedModel, AutoModelForSequenceClassification
from datasets import load_dataset
from torch.utils.data import DataLoader
import time
import psutil
import os
import warnings

warnings.filterwarnings("ignore")

In [16]:
tokenizer = get_tokenizer()
train_loader, val_loader = get_dataloaders(tokenizer)

Using the latest cached version of the dataset since Anthropic/hh-rlhf couldn't be found on the Hugging Face Hub


ValueError: Couldn't find cache for Anthropic/hh-rlhf for config 'default-data_dir=harmless-base'
Available configs in the cache: ['default', 'default-52e03caf22ec705f']

In [6]:
base_model = get_base_model()
print(f'Обучаемых параметров: {count_trinable_params(base_model)}')

lora_model = get_lora_model(lora_rank=4)
print(f'Обучаемых параметров с LoRA: {count_trinable_params(lora_model)}')


Обучаемых параметров: 1414647808
Обучаемых параметров с LoRA: 3355136


In [10]:
# lora_model.eval()
# base_model.eval()
train_model_exp(lora_model, tokenizer, train_loader, val_loader, max_length=32)

AttributeError: module 'torch.mps' has no attribute 'current_device'

In [ ]:
import time
def train_model_exp(model, texts, num_steps=100):
    # time and memory
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    opt = torch.optim.Adam(model.parameters(), lr=0.01)
    model.to(device)
    torch.cuda.reset_max_memory_allocated()
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.synchronize()
    start_time = time.perf_counter()
    for i in range(num_steps):
        inputs = tokenizer(texts, max_length=1024, return_tensors='pt', truncation=True).to(device)
        outputs = model(**inputs)
        next_word_logits = outputs.logits[:, : -1, :]
        true_next_tokens = inputs['input_ids'][:, 1:]
        loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))
        loss.backward()
        opt.step()
        opt.zero_grad()
        if i%10==0:
            print("Loss:", loss.item())
    torch.cuda.synchronize()
    time_elapsed = time.perf_counter() - start_time
    memory = torch.cuda.max_memory_allocated()
    return time_elapsed, memory


In [ ]:
inputs = tokenizer(batch['chosen'], max_length=4, truncation=True, padding='max_length', return_tensors='pt')
# print(inputs)
lora_model(**inputs)

In [ ]:
input_ids[attn_masks].max(), input_ids.min()

In [ ]:
res = base_model(torch.tensor(tokenized[0][0], dtype=int))
len(res)

In [ ]:
len(batch['chosen'])

In [ ]:
res[0].shape

### Supervised Fine-tuning

__Задание 2 (3 балла).__ Разбейте все примеры с "хорошими" ответами на запросы (все что идет до последнего "Assistant:") и ответы (все, начиная с последнего "Assistant:"). Дообучите модель [`pythia-1.4b`](https://huggingface.co/EleutherAI/pythia-1.4b) генерировать правильные ответы с помощью вашей LoRA. Одной эпохи вполне должно хватить для сходимости. Проверьте на нескольких случайных тестовых примерах, что модель ведет себя так, как надо.

In [ ]:
sft_train_loader, sft_val_loader = prepare_sft_dataloaders(train_loader, val_loader)

In [ ]:
# your code here

### Direct Preference Optimization

__Задание 3 (3 балла).__ Реализуйте DPO согласно [статье](https://arxiv.org/pdf/2305.18290) и дообучите SFT модель с предыдущего шага. Одной эпохи так же должно хватить, но можно обучать и дольше. Убедитесь, что модель начинает отдавать предпочтение хорошим ответам. Проведите анализ. Стали ли ответы лучше, чем у SFT модели? Всегда ли модель отвечает хорошо или иногда плохо? Насколько легко модель ломается при изменении промптов?